In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Whisper sub-token dump (top-50 per decoding step) for multiple languages (Beam Search with Rank)
================================================================================================
For each specified language:
  • Streams ≈600 s of Common Voice 17.0 audio
  • Forces the <|lang|> token so Whisper skips language-ID detection
  • Transcribes using BEAM SEARCH (beam_size=5, temperature=0.2)
  • Decodes step-by-step ALONG THE BEAM SEARCH PATH and records top-K candidate IDs + strings
  • Adds a 'chosen_rank' column indicating the rank of the chosen token in that step's top-K predictions
  • EVERY row carries `audio_path`, `ground_truth`, `full_transcription`,
    and `whisper_lang`, so merging is unnecessary
  • A sentinel row with step = -1 stores utterance-level metadata
  • One CSV per language is written to results_beam_600s/
"""

# ───────────────────────── Imports ────────────────────────────
import os
import shutil
import warnings
from pathlib import Path
import traceback

import torch
import whisper
import soundfile as sf
import pandas as pd
from datasets import load_dataset, Audio
from whisper.tokenizer import get_tokenizer
from tqdm import tqdm

# Silence a harmless HF warning about trust_remote_code
warnings.filterwarnings("ignore", category=UserWarning)

# ──────────────── Language configuration ─────────────────────────
LANGUAGES = {
    # Format: "language_name": {"num": index, "code": whisper_code, "cv": cv_code}
    # High-resource languages (more than ~10k hours)
    "german": {"num": 3, "code": "de", "cv": "de"},
    "spanish": {"num": 4, "code": "es", "cv": "es"},
    "french": {"num": 7, "code": "fr", "cv": "fr"},
    "portuguese": {"num": 9, "code": "pt", "cv": "pt"},
    "turkish": {"num": 10, "code": "tr", "cv": "tr"},
    
    # Medium-resource languages (between 100 and ~10k hours)
    "italian": {"num": 11, "code": "it", "cv": "it"},
    "dutch": {"num": 12, "code": "nl", "cv": "nl"},
    "swedish": {"num": 13, "code": "sv", "cv": "sv-SE"},
    "catalan": {"num": 14, "code": "ca", "cv": "ca"},
    "finnish": {"num": 15, "code": "fi", "cv": "fi"},
    "indonesian": {"num": 16, "code": "id", "cv": "id"},
    "vietnamese": {"num": 17, "code": "vi", "cv": "vi"},
    "romanian": {"num": 18, "code": "ro", "cv": "ro"},
    "danish": {"num": 19, "code": "da", "cv": "da"},
    "norwegian": {"num": 20, "code": "no", "cv": "nn-NO"},
    "czech": {"num": 21, "code": "cs", "cv": "cs"},
    "hungarian": {"num": 22, "code": "hu", "cv": "hu"},
    "yoruba": {"num": 23, "code": "yo", "cv": "yo"},
    
    # Low-resource languages (less than 100 hours)
    "welsh": {"num": 39, "code": "cy", "cv": "cy"},
    "lithuanian": {"num": 35, "code": "lt", "cv": "lt"},
    "latvian": {"num": 43, "code": "lv", "cv": "lv"},
    "azerbaijani": {"num": 46, "code": "az", "cv": "az"},
    "estonian": {"num": 49, "code": "et", "cv": "et"},
    "basque": {"num": 52, "code": "eu", "cv": "eu"},
    "albanian": {"num": 59, "code": "sq", "cv": "sq"},
    "swahili": {"num": 60, "code": "sw", "cv": "sw"},
    "maltese": {"num": 85, "code": "mt", "cv": "mt"},
    "uzbek": {"num": 79, "code": "uz", "cv": "uz"},
}

# ───────────── Global configuration ──────────────────────────────
TOP_K = 50
MAX_DECODER_INPUT_TOKENS = 448  # Max tokens for decoder input sequence (context window)
TARGET_SEC = 600  # 10 minutes of audio per language
BEAM_SIZE = 5
TEMPERATURE = 0.2

OUT_DIR = Path(f"results_beam_{TARGET_SEC}s")
TEMP_DIR = OUT_DIR / "tmp_audio"
ERROR_LOG = OUT_DIR / "errors.log"

OUT_DIR.mkdir(exist_ok=True, parents=True)
TEMP_DIR.mkdir(exist_ok=True, parents=True)
if ERROR_LOG.exists():
    ERROR_LOG.unlink()

# ─────────────── Whisper model (shared) ──────────────────────────
print(f"Loading Whisper (large-v2) on device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("large-v2", device=device)
print("Whisper model loaded.")


# ─────────── Helper: stream ~TARGET_SEC of audio ─────────────────
def load_some_cv_audio(cv_code: str, seconds: int):
    """
    Load approximately `seconds` worth of audio from Common Voice 17.0.
    Tries validated, train, and test splits in order.
    """
    splits = ["validated", "train", "test"]
    for split in splits:
        try:
            ds = load_dataset(
                "mozilla-foundation/common_voice_17_0",
                cv_code,
                split=split,
                streaming=True,
                trust_remote_code=True,
            ).cast_column("audio", Audio(16000))
            
            batch, dur = [], 0.0
            for ex in ds:
                raw = ex["path"]
                ex["cv_path"] = f"{cv_code}_{split}/{Path(raw).name}"
                arr = ex["audio"]["array"]
                dur += len(arr) / 16000
                batch.append(ex)
                if dur >= seconds:
                    return batch

        except Exception as e:
            print(f"  ⚠️ Error loading {cv_code} {split}: {str(e)}")
            continue
    
    print(f"No audio found for {cv_code} after trying all splits.")
    return []


def log_error_to_file(language, error_msg):
    """Log error details to the error log file."""
    with open(ERROR_LOG, 'a', encoding='utf-8') as f:
        f.write(f"Language: {language}\nError: {error_msg}\n\n")


# ───────────────────── Main loop ─────────────────────────────────
total_langs_to_process = len(LANGUAGES)
processed_langs_count = 0
successful_langs_count = 0
failed_langs_count = 0

for lang_name, info in sorted(LANGUAGES.items(), key=lambda x: x[1]["num"]):
    processed_langs_count += 1
    lang_num = info["num"]
    lang_code = info["code"]
    cv_code = info["cv"]

    print(f"\n─── [{lang_num:03d}/{total_langs_to_process}] {lang_name.upper()} ({lang_code} / {cv_code}) ───────────")

    try:
        samples = load_some_cv_audio(cv_code, TARGET_SEC)
        if not samples:
            print(f"⚠️  No audio found for {lang_name} – skipping.")
            log_error_to_file(lang_name, "No audio found after trying all splits.")
            failed_langs_count += 1
            continue

        total_dur = sum(len(s["audio"]["array"]) / 16000 for s in samples)
        print(f"✓ Processing {len(samples)} clips, {total_dur:.1f}s total for {lang_name}")

        tok = get_tokenizer(multilingual=True, language=lang_code, task="transcribe")
        lang_token_id = tok.encode(f"<|{lang_code}|>", allowed_special="all")[0]
        transcribe_token_id = tok.encode("<|transcribe|>", allowed_special="all")[0]
        sot_token_id = tok.sot

        rows = []
        for clip_id, ex in enumerate(tqdm(samples, desc=f"Decoding {lang_name}")):
            cv_path = ex["cv_path"]
            ground_truth = ex.get("sentence", "").strip()

            wav_path = TEMP_DIR / f"{lang_name}_{cv_code}_{clip_id}.wav"
            sf.write(wav_path, ex["audio"]["array"], 16000, subtype='PCM_16')

            # Perform transcription using beam search
            res = model.transcribe(
                str(wav_path),
                language=lang_code,
                task="transcribe",
                beam_size=BEAM_SIZE,
                temperature=TEMPERATURE,
                verbose=None
            )
            
            full_text = res["text"].strip()
            lang_guess = res["language"]

            # Extract beam search output tokens
            beam_search_output_tokens = []
            if "segments" in res and res["segments"]:
                for segment in res["segments"]:
                    beam_search_output_tokens.extend(segment['tokens'])
            else:
                print(f"Warning: Segments not found for clip {clip_id} ({lang_name}). Re-tokenizing full_text.")
                beam_search_output_tokens = tok.encode(full_text)
            
            # Ensure EOT is included if missing
            if not beam_search_output_tokens or beam_search_output_tokens[-1] != tok.eot:
                if len(beam_search_output_tokens) < MAX_DECODER_INPUT_TOKENS - 1:
                    beam_search_output_tokens.append(tok.eot)
                else:
                    beam_search_output_tokens[-1] = tok.eot

            # Add sentinel row with utterance-level metadata
            rows.append({
                "clip": clip_id,
                "step": -1,
                "audio_path": cv_path,
                "ground_truth": ground_truth,
                "full_transcription": full_text,
                "whisper_lang": lang_guess,
                "chosen_rank": None
            })

            # Prepare encoder output
            audio_features = whisper.load_audio(str(wav_path))
            mel_segment = whisper.log_mel_spectrogram(whisper.pad_or_trim(audio_features)).to(device)
            encoder_output = model.encoder(mel_segment.unsqueeze(0))

            current_decoder_input_token_ids = [sot_token_id, lang_token_id, transcribe_token_id]

            # Step through each token in the beam search output
            for step_idx, actual_next_token_id in enumerate(beam_search_output_tokens):
                if len(current_decoder_input_token_ids) >= MAX_DECODER_INPUT_TOKENS:
                    print(f"Warning: Decoder input sequence for clip {clip_id}, step {step_idx} "
                          f"reached {len(current_decoder_input_token_ids)} tokens. Breaking step-by-step.")
                    break
                
                decoder_input_tensor = torch.tensor([current_decoder_input_token_ids], device=device)
                
                with torch.no_grad():
                    logits_for_next_token = model.decoder(decoder_input_tensor, encoder_output)[0, -1]
                
                probabilities_for_next_token = torch.softmax(logits_for_next_token, dim=-1)
                top_k_probs, top_k_ids = torch.topk(probabilities_for_next_token, TOP_K)

                # Get probability of the chosen token
                chosen_token_prob = 0.0
                if actual_next_token_id < probabilities_for_next_token.shape[0]:
                    chosen_token_prob = probabilities_for_next_token[actual_next_token_id].item()
                
                # Calculate rank of the chosen token
                chosen_rank = TOP_K + 1  # Default if not in top_k
                try:
                    chosen_rank = top_k_ids.tolist().index(actual_next_token_id) + 1
                except ValueError:
                    pass  # Remains TOP_K + 1

                row_data = {
                    "clip": clip_id,
                    "step": step_idx,
                    "audio_path": cv_path,
                    "ground_truth": ground_truth,
                    "full_transcription": full_text,
                    "whisper_lang": lang_guess,
                    "chosen_id": actual_next_token_id,
                    "chosen_prob": chosen_token_prob,
                    "chosen_rank": chosen_rank,
                }
                
                # Add top-K predictions
                for k_val in range(TOP_K):
                    if k_val < len(top_k_ids):
                        token_id_k = top_k_ids[k_val].item()
                        row_data[f"top{k_val+1}_id"] = token_id_k
                        row_data[f"top{k_val+1}_txt"] = tok.decode([token_id_k])
                        row_data[f"top{k_val+1}_prob"] = float(top_k_probs[k_val].item())
                    else:
                        row_data[f"top{k_val+1}_id"] = None
                        row_data[f"top{k_val+1}_txt"] = ""
                        row_data[f"top{k_val+1}_prob"] = 0.0
                
                rows.append(row_data)

                current_decoder_input_token_ids.append(actual_next_token_id)
                if actual_next_token_id == tok.eot:
                    break
            
            if wav_path.exists():
                wav_path.unlink()

        # Write results to CSV
        csv_path = OUT_DIR / f"{lang_num:03d}_{lang_name}_subtoken_beam.csv"
        df = pd.DataFrame(rows)
        
        # Reorder columns for better readability
        cols = list(df.columns)
        if "chosen_rank" in cols:
            rank_idx = cols.index("chosen_rank")
            prob_idx = cols.index("chosen_prob")
            if rank_idx > prob_idx + 1:
                cols.insert(prob_idx + 1, cols.pop(rank_idx))
                df = df[cols]
        
        df.to_csv(csv_path, index=False, encoding='utf-8')
        print(f"✓ CSV written → {csv_path}")
        successful_langs_count += 1

    except Exception as e:
        detailed_error_msg = f"{type(e).__name__}: {str(e)}\n{traceback.format_exc()}"
        print(f"❌ Error processing {lang_name}: {detailed_error_msg}")
        log_error_to_file(lang_name, detailed_error_msg)
        failed_langs_count += 1
        
        # Clean up temp file if error occurred
        if 'wav_path' in locals() and wav_path.exists():
            wav_path.unlink()
        continue

# ───────────── Cleanup ───────────────────────────────────────────
if TEMP_DIR.exists():
    try:
        shutil.rmtree(TEMP_DIR)
        print(f"✓ Temporary directory {TEMP_DIR} removed.")
    except OSError as e:
        print(f"Error removing temporary directory {TEMP_DIR}: {e.strerror}")

# ───────────── Summary ───────────────────────────────────────────
print(f"\n✓ All {processed_langs_count}/{total_langs_to_process} language tasks attempted.")
print(f"  Successfully processed: {successful_langs_count} languages.")
if failed_langs_count > 0:
    print(f"  Failed to process: {failed_langs_count} languages (see {ERROR_LOG} for details).")
print(f"  Output files are in: {OUT_DIR}")


/home/siyu/.conda/envs/whisper_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Whisper (large-v2) on device: NVIDIA A100-PCIE-40GB
Whisper model loaded.

─── [003/28] GERMAN (de / de) ───────────


Reading metadata...: 917350it [00:22, 41317.27it/s]


✓ Processing 98 clips, 601.7s total for german


Decoding german: 100%|██████████| 98/98 [03:10<00:00,  1.94s/it]


...

✓ CSV written → results_beam_600s/079_uzbek_subtoken_beam.csv

─── [085/28] MALTESE (mt / mt) ───────────


Reading metadata...: 6442it [00:00, 11225.47it/s]


✓ Processing 110 clips, 600.7s total for maltese


Decoding maltese: 100%|██████████| 110/110 [05:53<00:00,  3.22s/it]


✓ CSV written → results_beam_600s/085_maltese_subtoken_beam.csv
✓ Temporary directory results_beam_600s/tmp_audio removed.

✓ All 28/28 language tasks attempted.
  Successfully processed: 28 languages.
  Output files are in: results_beam_600s
